## new feature test

In [1]:
import pickle
import numpy as np
import tensorflow as tf

from math import sqrt
from tqdm import tqdm
from copy import deepcopy
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

C:\Users\home\Anaconda3\envs\TF\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
with open("raw.pickle","rb") as f:
    data = pickle.load(f)

with open("matrix.pickle","rb") as f:
    matrix = pickle.load(f)

print (len(matrix.keys()))

19570


In [3]:
in_matrix = {}
for k in matrix:
    for v in matrix[k]:
        if v in in_matrix:
            in_matrix[v].add(k)
        else:
            in_matrix[v] = set([k])

In [4]:
def hop_point(a, b, in_matrix, matrix, normalize=True):
    cnt = 0
    for hop in matrix[a]:
        if hop in in_matrix[b]:
            cnt += 1
    if normalize:
        cnt /= len(matrix[a])
    return cnt

def kNN(a, b, in_matrix, matrix):
    wa_out = 1/sqrt(1+len(matrix[a]))
    wb_in = 1/sqrt(1+len(in_matrix[b]))
    return wa_out + wb_in

In [5]:
tf.reset_default_graph()

# give id to node with outdegree > 0
valid_nodes = list(matrix.keys())
v2id = {}
for idx, v in enumerate(valid_nodes):
    v2id[v] = idx
print(len(v2id))

# computation graph for rooted page rank
d = 0.5
walk = 5
A_ = tf.placeholder(tf.float32, [len(v2id)+1, len(v2id)+1])
x_ = tf.placeholder(tf.float32, [len(v2id)+1,1])
prob = tf.matmul(A_, x_)
for i in range(walk-1):
    prob = (1 - d) * prob + d * tf.matmul(A_, prob)

19570


In [6]:

# filter sinks with outdegree 0
m = {}
tB = {}
for vi in matrix:
    valid = set()
    for vj in matrix[vi]:
        tB[vj] = tB.get(vj,set())
        tB[vj].add(v2id[vi])
        if vj in v2id:
            valid.add(vj)
    m[v2id[vi]] = [v2id[v] for v in valid]

print(len(m))

tm = np.zeros([len(v2id)+1, len(v2id)+1])
for vi in m:
    if len(m[vi])>0:
        tm[vi,m[vi]] = 1/len(m[vi])

19570


In [7]:
import time


def pagerank(a, b, matrix):
        
    s1 = time.clock()
    
    ta = v2id[a]
    A = tm.copy()
    
    aset = deepcopy(tB[b])
    if ta in aset:
        aset.remove(ta)
    
    for vi in aset:
        if not vi == ta:
            tempv = 1/(len(m[vi])+1)
            A[vi,m[vi]] = tempv
            A[vi,-1]=tempv
    
    
    
    x = np.zeros(len(v2id)+1)
    x[v2id[a]] = 1
    """
    for i in range(walk):
        x = (1 - d) * x + d * A.T @ x
    """
    s2 = time.clock()
    
    x = sess.run(prob, feed_dict={A_:A.T, x_:np.expand_dims(x,-1)})
    
    end = time.clock()
    #print (s2-s1,end-s2)
    
    if b in matrix:
        return x[v2id[b]][0]
    else:
        return x[-1][0]

In [8]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.device("/gpu:0"):
    with tf.Session(config=config) as sess:
        train_ = {}
        for i, a, b, label in tqdm(data, ascii=True):
            #if a == 1818439:
            #    continue
            #hop = hop_point(a, b, in_matrix, matrix)
            #knn = kNN(a, b, in_matrix, matrix)
            pr = pagerank(a, b, matrix)
            train_[i]=[pr, label]
        #train_ = np.array(train_)

  0%|                                                                                        | 0/20000 [00:00<?, ?it/s]

1.4871397402597404 4.886293887445888


  0%|                                                                             | 1/20000 [00:06<36:59:48,  6.66s/it]

1.1863380779220778 4.624977731601731


  0%|                                                                             | 2/20000 [00:12<35:25:32,  6.38s/it]

...

  0%|2                                                                           | 64/20000 [06:25<33:20:41,  6.02s/it]

1.0845922077921841 4.307938077922074


  0%|2                                                                           | 65/20000 [06:31<33:18:41,  6.02s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\home\Anaconda3\envs\TF\lib\site-packages\IPython\core\interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-bf7a4da6a017>", line 12, in <module>
    pr = pagerank(a, b, matrix)
  File "<ipython-input-7-b1623d3107e9>", line 31, in pagerank
    x = sess.run(prob, feed_dict={A_:A.T, x_:np.expand_dims(x,-1)})
  File "C:\Users\home\Anaconda3\envs\TF\lib\site-packages\tensorflow\python\client\session.py", line 901, in run
    if run_metadata:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\home\Anaconda3\envs\TF\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Trace

KeyboardInterrupt: 

In [ ]:

with open("1W.pickle","wb") as f:
    pickle.dump(train_, f)

In [8]:
#with tf.device("/gpu:0"):
#    with tf.Session() as sess:
#        test_pagerank = []
#        for a, b in tqdm(test_pair, ascii=True):
#            #hop = hop_point(a, b, in_matrix, matrix)
#            #knn = kNN(a, b, in_matrix, matrix)
#            pr = pagerank(a, b, matrix)
#            test_pagerank.append([pr])
#        test_pagerank = np.array(test_pagerank)

In [9]:
test_pagerank.shape

NameError: name 'test_pagerank' is not defined

In [ ]:
to_disc = {}
for i in range(len(test_pagerank)):
    to_disc[i] = test_pagerank[i][0]
with open("pagerank_test.pickle","wb") as f:
    pickle.dump(to_disc, f)

In [ ]:
train, test = train_test_split(train_, train_size=0.8)
X_train, y_train = train[:,:-1], train[:,-1]
X_test, y_test = test[:,:-1], test[:,-1]

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(roc_auc_score(y_test, np.squeeze(model.predict_proba(X_test)[:,1])))

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(roc_auc_score(y_test, np.squeeze(model.predict_proba(X_test)[:,1])))

In [ ]:
model = RandomForestClassifier(100)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(roc_auc_score(y_test, np.squeeze(model.predict_proba(X_test)[:,1])))

In [ ]:
y_pred_prob = model.predict_proba(test_pagerank)
with open("pred_pagerank.csv","w") as f:.
    f.write("Id,Prediction\n")
    for i in range(1,len(y_pred_prob)+1):
        f.write(str(i) + "," + str(y_pred_prob[i-1][1]) + "\n")

In [ ]:
print(train_)